In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df_train=pd.read_csv('train_final.csv',index_col=False)
labels=df_train[['784']]

In [3]:
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
np.random.seed(1212)
import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input, Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.common.image_dim_ordering()

Using TensorFlow backend.


'tf'

In [5]:
labels=np.array(labels)

In [6]:
from keras.utils.np_utils import to_categorical
cat=to_categorical(labels,num_classes=13)

In [7]:
print(cat[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [8]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
l=[]
for i in range(47504):
    l.append(np.array(df_train[i:i+1]).reshape(1,28,28))

In [10]:
np.random.seed(7)

In [11]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu', data_format='channels_first'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(13, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
from keras.models import model_from_json

In [13]:
model.fit(np.array(l), cat, epochs=10, batch_size=200,shuffle=True,verbose=1)

Epoch 1/10
47504/47504 [==============================] - 25s 525us/step - loss: 1.7141 - accuracy: 0.5713
Epoch 2/10
47504/47504 [==============================] - 27s 562us/step - loss: 0.4506 - accuracy: 0.8632
Epoch 3/10
47504/47504 [==============================] - 25s 534us/step - loss: 0.2469 - accuracy: 0.9258
Epoch 4/10
47504/47504 [==============================] - 24s 515us/step - loss: 0.1674 - accuracy: 0.9490
Epoch 5/10
47504/47504 [==============================] - 24s 511us/step - loss: 0.1281 - accuracy: 0.9604
Epoch 6/10
47504/47504 [==============================] - 24s 512us/step - loss: 0.1001 - accuracy: 0.9690
Epoch 7/10
47504/47504 [==============================] - 24s 514us/step - loss: 0.0823 - accuracy: 0.9745
Epoch 8/10
47504/47504 [==============================] - 25s 517us/step - loss: 0.0670 - accuracy: 0.9793
Epoch 9/10
47504/47504 [==============================] - 28s 599us/step - loss: 0.0562 - accuracy: 0.9826
Epoch 10/10
47504/47504 [============

In [14]:
model_json = model.to_json()
with open("model_final.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_final.h5")